In [ ]:
import torch # pytorch basic package
from torch import nn # neural net 
from torch.utils.data import DataLoader, Dataset # to work with data
from torchvision import datasets # built-in data
from torchvision.transforms import ToTensor # to convert nparrays/images into pytorch tensors
from torchvision import models
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from nn_utils import train, test

In [ ]:
torch.manual_seed(0) # for reproducibility
device = "cuda" if torch.cuda.is_available() else "cpu" # set GPU

## The CIFAR100 dataset
Let's experiment with a new dataset with many more images and more classes:

In [ ]:
# Download training data from open datasets.
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

trainloader = DataLoader(training_data, batch_size=8, shuffle=True)
testloader = DataLoader(test_data, batch_size=8, shuffle=False)

In [ ]:
print(len(test_data))

In [ ]:
resnet_model = models.resnet50(weights=None)

num_classes = 100

print(resnet_model)

In [ ]:
resnet_model.fc = nn.Linear(in_features = resnet_model.fc.in_features, out_features = num_classes)
resnet_model = resnet_model.to(device)

print(resnet_model)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epochs = 10
train_losses = []
test_losses = []
for t in range(epochs):
    print(f'EPOCH {t} --------------------')
    train_loss = train(trainloader, resnet_model, loss, optimizer, 1000, device)
    test_loss = test(t, testloader, resnet_model, loss, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

In [ ]:
resnet_model_pretrained = models.resnet50(weights="IMAGENET1K_V1")
resnet_model_pretrained.fc = nn.Linear(in_features = resnet_model_pretrained.fc.in_features, out_features = num_classes)
resnet_model_pretrained = resnet_model_pretrained.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_model_pretrained.parameters(), lr=0.001, momentum=0.9)

epochs = 15
train_losses = []
test_losses = []
for t in range(epochs):
    print(f'EPOCH {t} --------------------')
    train_loss = train(trainloader, resnet_model_pretrained, loss, optimizer, 1000, device)
    test_loss = test(t, testloader, resnet_model_pretrained, loss, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

## AUGMENTATIONS

In [ ]:
import torchvision.transforms as transforms

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.GaussianBlur(kernel_size=3)
])

In [ ]:
# Download test data from open datasets.
test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# Download training data from open datasets.
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    download=True,
    transform=transform_train,
)

trainloader = DataLoader(training_data, batch_size=8, shuffle=True)
testloader = DataLoader(test_data, batch_size=8, shuffle=False)

In [ ]:
resnet_model_pretrained = models.resnet50(weights="IMAGENET1K_V1")
resnet_model_pretrained.fc = nn.Linear(in_features = resnet_model_pretrained.fc.in_features, out_features = num_classes)
resnet_model_pretrained = resnet_model_pretrained.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_model_pretrained.parameters(), lr=0.001, momentum=0.9)

epochs = 15
train_losses = []
test_losses = []
for t in range(epochs):
    print(f'EPOCH {t} --------------------')
    train_loss = train(trainloader, resnet_model_pretrained, loss, optimizer, 1000, device)
    test_loss = test(t, testloader, resnet_model_pretrained, loss, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

## HOMEWORK

1. Try different built-in models to beat ResNet-50 in the CIFAR-100 classification task! Plot the different loss curves onto each other so that the differences are visible.
2. Try different augmentations (less, more, tune the parameters) for the best model and see what improves the accuracy and what doesn't.
3. Use your best model. Take 1000 random images from the test set and save the correctly predicted images into one directory and the misclassified ones into another directory. The images should have text above them with the ground truth and predicted classes (labels_map available online).